# Project Status and Work Log

## Initial Approach and Thoughts
* My initial approach focused on training a fast word based tagging model. A full set of feature extraction and hyper parameter tuning was conducted
  * However the results were un satisfactory (see the non RNN  so I switched to an RNN model)
  * Also note that the mongo collections have been renamed to differentiate this fix - metrics_coref_word_tagger are the original word-based tagging model work, which are valid
* Switching to the RNN model, I made some initital mistakes:
 1. I didn't have it choose the Anaphora tag but instead chose the most common tag (which isn't always the anaphora tag)
 2. There seems to be some issues with the way the initial RNN's trained, e.g. compare the numbers under the Anaphora.data_points. This is correct in the word tagging model and the later RNN tagger (see "_fixed") but not in the initial RNN tagger work
* To remedy this I switched to newer code, copied from different notebooks, and the results of that are seen under the "coref_new_fixed" collection
* I also moved to having the models spit out tagged data points, and then re-computing the metrics directly from that data. This then allows us to interrogate those predictions at a later date as needed, not just the raw numbers

## Mongo Collection Naming
-  9/15/2018 Deleted - metrics_coref_broken
        - Initial RNN tagger work with non-optimal model
        
- metrics_coref_old


## Adjusting the Bratt Essay Parsing Logic to Resolve Anaphora Tags
* Next I adjusted the BrattEssay file to resolve the anaphora tags with their antecedents when provided
* These get resolved as Anaphora:[{code}] where {code} is one of the 13 or 9 concept codes, e.g. Anaphora:[50]
* Analysis of how anaphora tags are initially tagged -  see 'Examine at How Anaphora Tags are Tagged to Inform Essay Parser Changes' 

## Next Steps
* Hyper parameter tune the fixed, binary RNNs
 - Done for CB, but no results in mongo for SC (do i need to sync to old mac?)
* Validate the bratt parser logic
    * See "Test Bratt Essay Changes to include Anaphora Tags"
* Load the predictions and reconcile with the co-ref parser output
* Evaluate 2 things:
    1. Taking the predicted tags, look for intersections in co-ref output, and evaluate accuracy of the resolved anaphora concept codes
    2. Use the stanford co-reference parser alone to implement this logic

## Questions 
- Are the Co-references only present for concept codes? 
 - No (and see ans to next qu)
 - Also, some anaphora tags have no antecedent
- Are they only present for concept codes that form part of a causal relation?
 - No, not all anaphora tags have causal relations

##  Measure of Success - Considerations
  * Accuracy at detecting anaphora tags
  * Accuracy at detecting anaphora tags and correctly resolving the associated concept(s)
      1. Using the ML model's predictions to filter
      2. Using the stanford output alone
  * Impact on other metrics when incorporated into a single solution?